# DSCI 100 Term Project
- name: Geoff Acabado
- student number: 59285189

## Introduction
For this project, we will be looking how does different player characteristics correlate to thir probability of being subscribed to a game news paper. A more extensive guide and analysis can be found in the actual report. This ipynb file only cointains the code necessary to achieve the 

In [ ]:
#Some libraries to install
library(repr)
library(tidyverse)
library(tidymodels)
options(repr.matrix.max.rows = 10)
set.seed(9999)

### Tidying the data

In [ ]:
#Code 1.1
# install the data needed
player_data = read_csv("players.csv")
player_data

### Data explorations

In [ ]:
#Code 1.2
#Lets explore relationships between hours played, age and whether they are subscibed to game newspaper
options(repr.plot.height = 7, repr.plot.width = 10)
player_plot = player_data |>
    ggplot(aes(y = played_hours	, x = Age, color = subscribe)) +
    geom_point() +
    labs(
        title = "Hours played Vs Age of Player",
        y = "Hours played by player",
        x = "Age of player (Years)",
        colour = "Player is subscribed" ) +
    theme(text = element_text(size = 16), plot.title = element_text(hjust = 0.5))
player_plot

#### Figure 1.1
The plot above shows the relationship between the age of the player and the number of hours they have played Minecraft. It can be observed that most of the data points are concentrated at lower hours played. It might be beneficial to use a logarithmic scale on the y-axis to see how data points differ from each other especially for lower hours played values.

#### Code 1.2 Warning
The code above gave us a warning message indicating that there are 2 datapoints that have missing values or are out of range. Let's look for those values.

In [ ]:
missing_values = player_data |>
    filter(is.na(Age))
missing_values

It looks like we found the 2 missing values so there is no need to see if there are any values out of scale. Lets proceed to making the scale of the data better

In [ ]:
#Code 1.3
#Lets explore relationships between hours played, age and whether they are subscibed to game newspaper
player_plot_1 = player_data |>
    ggplot(aes(y = played_hours	, x = Age, color = subscribe)) +
    geom_point() +
    scale_y_log10() +
    labs(
        title = "Scaled Hours played Vs Age of Player",
        y = "Hours played by player (lograithmic scale)",
        x = "Age of player (Years)",
        colour = "Player is subscribed" ) +
    theme(text = element_text(size = 15), plot.title = element_text(hjust = 0.5))
player_plot_1



#### Figure 1.2
From the figure above, it is easy to see that there are no obvious pattern between between hours played by the player and their age. Both variables also do not seem to be correlated with whether the players are subscribed or not. It is intersting to see however that everyone who played for 10 hours or more are all subscribed to a game newspaper. Another observation we see is that most players are arround 20 years old indicating that this is minecraft is popular among this demographic

#### Code 1.3 Warning
There was a warning that the logarithmic transformation turned the some 0s to infinity but it looks like it did not affect the purpose of our visualization that much. All of the supposedly 'negatively infinite' values remained at the bottom of the graph. 

### Data Analysis Plan

- Build a classification model that lets us categorize a player according to whether or not they are subscribed in a gaming newspaper
- Use their hours played in the game as well as their age as predictors
- Split the data into training and testing data
- Use a k-nn algorythm with a 5 fold cross validation to verify what the best K-value should be. 
- A K value that will give highest accuracy when used on testing data will be used to build the final model
- Use different metrics such as accuracy, precision, and recall to determine if data is useful in real world

### Model Tuning
Please refer to the actual report 

In [ ]:
#Code 2.1
# Clean up data
player_data_clean = player_data |>
    select(subscribe, played_hours, Age) |>
    mutate(subscribe = as.factor(subscribe))

In [ ]:
#Code 2.2
#split the data
player_data_split = initial_split(player_data_clean, prop = 0.7, strata = subscribe)
player_data_training = training(player_data_split)
player_data_testing = testing(player_data_split)

In [ ]:
#Code 2.3
# Tuning the model with 5 fold cross validation
player_data_vfold = vfold_cv(player_data_training, v = 5, strata = subscribe)

In [ ]:
#Code 2.4
#make recipe
player_data_recipe = recipe(subscribe ~., data = player_data_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

In [ ]:
#Code 2.5
#Tune model
knn_spec = nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

In [ ]:
#Code 2.6
#Get results
k_vals = tibble(neighbors = 1:25)
knn_results = workflow() |>
    add_recipe(player_data_recipe) |>
    add_model(knn_spec) |>
    tune_grid(resamples = player_data_vfold, grid = k_vals) |>
    collect_metrics() |>
    filter(.metric == "accuracy", !is.na(mean))

In [ ]:
#Code 2.7
knn_results_plot = knn_results |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Nearest Neighbor" , y = "Mean Accuracy")+
    theme(text = element_text(size  = 12))
knn_results_plot

#### Figure 2.1
This graph shows mean accuracy for different values of K. We see that there is a local maximum at K = 5 to 8 (excluding k = 6) which are probably the best K for this data set. Let's just chose 7. We see that after K = 10, the accuracy increases again but that is probably just because there are a lot more people subscribed than not subscribed to news paper. As K increaases, the alogrythm will just classify any point as "TRUE" which results in high accuracy but is not very useful in this context.

#### Code 2.6 Warning
During hyperparameter tuning, warnings related to row mismatches were encountered when certain K values exceeded the number of available observations within some cross-validation folds. These cases were automatically handled by the tuning procedure, and valid results were obtained for all feasible parameter combinations. Therefore, these warnings did not affect the final analysis.

### Final Model Building and Evaluation

In [ ]:
#Code 3.1
# Create final mode
final_spec = nearest_neighbor(weight_func = "rectangular", neighbors = 7) |>
  set_engine("kknn") |>
  set_mode("classification")

final_fit <- workflow() |>
    add_recipe(player_data_recipe) |>
    add_model(final_spec) |>
    fit(data = player_data_training)


#Evaluate final model
test_predictions <- predict(final_fit, player_data_testing) |>
  bind_cols(player_data_testing)

# Step 2: Calculate evaluation metrics
eval_metrics <- bind_rows(
  accuracy(test_predictions, truth = subscribe, estimate = .pred_class),
  precision(test_predictions, truth = subscribe, estimate = .pred_class),
  recall(test_predictions, truth = subscribe, estimate = .pred_class),
)

eval_metrics

### Discussion and Conclusions (See report for more extensive analysis)
We see that accuracy is only 63.33% which is only a little bit better than random chance. We also see that precision is 35% and recall is 43.75% which are both bad. These values suggest that the model does not perform reliably, especially when it comes to correctly identifying positive cases. The low precision means that many of the predicted positives were actually incorrect, while the low recall shows that the model missed a large portion of the actual positive cases. One possible reason for this could be an imbalance in the data, where one class is much more common than the other. To improve the model, further testing with different settings and better preparation of the data may be helpful. Exploring other classification methods in the future may also lead to better performance.
